データの確認

In [1]:
#ライブラリのインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import string
import jctconv
import emoji
import re

In [2]:
#データの読み込み
df = pd.read_csv('./all_rakuma.csv')
df.head()

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,comments,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict
0,2019/10/08 23:39:51,https://item.fril.jp/055f4080a2a0ae275db6ba7be...,Epiphone LTD Korina Explorer Bass,40000,Epiphone,楽器 > ベース > エレキベース,109,0,0,1-2日後,NaN,2000-01-01,0,岐阜県,着払い,https://fril.jp/shop/42df208014145c438f96e01f5...,なし,https://img.fril.jp/img/259586636/l/734809353....,コリーナボディ ブラックピックガード ゴールドパーツの豪華なエクスプローラー コリーナ ...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', 'ブランド':..."
1,2019/10/08 23:39:51,https://item.fril.jp/896a1d0197d05cf8431c136d9...,TVB-100 (SUN),38000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,"['TOKYO GUITARS', '本モデルを含め、TOKYO GUITARS工房にて製造...",2019/08/31,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223532100/l/635334018....,Tokyo Guitars モデル名 TVB 100 SUN Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
2,2019/10/08 23:39:51,https://item.fril.jp/6bdb6b4dd0f6c518938b94ec2...,TRB-4000 (BL),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294381/l/634674588....,Tokyo Guitars モデル名 TRB 4000 BL Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
3,2019/10/08 23:39:51,https://item.fril.jp/b89c82258515163e5aa36aa48...,TRB-4000 (FG),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294160/l/634673947....,Tokyo Guitars モデル名 TRB 4000 FG Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
4,2019/10/08 23:39:51,https://item.fril.jp/cf904ef67c6cec09a7465e7f4...,TB-100 (BRWN),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/216145047/l/614817207....,Tokyo Guitars モデル名 TB 100 BRWN Tokyo Guit...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."


In [7]:
df.describe()

,price,good,bad,normal,good_counts
count,1.220700e+05,122070.000000,122070.000000,122070.000000,122070.0
mean,1.314400e+04,186.026419,0.544696,1.928770,0.0
std,4.780766e+04,267.706850,2.087027,5.201109,0.0
min,3.000000e+02,0.000000,0.000000,0.000000,0.0
25%,1.500000e+03,30.000000,0.000000,0.000000,0.0
50%,4.180000e+03,114.000000,0.000000,0.000000,0.0
75%,1.280000e+04,229.000000,0.000000,2.000000,0.0
max,2.000000e+06,6646.000000,63.000000,169.000000,0.0


データの整理

In [151]:
#カラムの削除
data = df.drop(columns=['date', 'item_url', 'good', 'bad', 'normal', 'ship_days', 'comments', 
                        'last_log_date', 'good_counts', 'prefecture', 'seller_url', 'size', 
                        'picture_url', 'details_dict'])

In [152]:
#カテゴリとブランドのユニーク値の数を確認
print(data.category.nunique())
print(data.brand.nunique())

1012
1637


In [153]:
#カテゴリとブランドの一項目当たりの数を数える
ctg_data = data.category.value_counts().reset_index()
br_data = data.brand.value_counts().reset_index()

In [154]:
#精度を挙げるために数の少ない項目を削除
ca500_data = ctg_data[ctg_data.category > 1000]
br100_data = br_data[br_data.brand > 150]

In [155]:
#mergeしやすいようにカラムの名前を変更
ca500_data = ca500_data.rename(columns={'index': 'index_ca'})
br100_data = br100_data.rename(columns={'index': 'index_br'})

In [156]:
#項目のみのデータフレームにする
ca500_data_idx = ca500_data[['index_ca']]
br100_data_idx = br100_data[['index_br']]

In [157]:
#データを結合し元のデータから項目を削除
new_data = data.merge(ca500_data_idx, left_on='category', right_on='index_ca')
new_data = new_data.merge(br100_data_idx, left_on='brand', right_on='index_br')
new_data = new_data.drop(columns=['index_ca', 'index_br'])
new_data.head()

,title,price,brand,category,delivery_fee,description
0,テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品,2500,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,送料込,テレカ 未使用 50度数 レア 仮面ライダー 誕生20周年 レーザーディスク販売時の販促...
1,送料無料 新品 DVD Perfume WORLD TOUR 1st 初回,4800,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,送料込,新品 DVD Perfume WORLD TOUR 1st 初回プレス盤 ご覧頂きあり...
2,☆ウルトラマン☆レーザーディスク ジャンク？,1980,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,送料込,ウルトラマンのレーザーディスクです 再生機器が無いのと 未開封のようですので現状渡しで...
3,MAISHA （マイシャ）Sadao Watanabe 渡辺貞夫★LD,2600,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,着払い,ご覧いただきありがとうございます MAISHA マイシャ 1985年 Sadao...
4,LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA,3000,Laserdiscs (R),エンタメ/ホビー > エンタメその他 > その他,着払い,ご覧いただきありがとうございます PARKER S MOOD パーカーズ ムード LI...


In [158]:
#new_dataのカテゴリとブランドのユニーク値の数を確認
print(new_data.category.nunique())
print(new_data.brand.nunique())

22
93


データの前処理

In [159]:
#テキストから取り除く絵文字を指定
emoji1 = "".join(emoji.UNICODE_EMOJI['en'].keys())
emoji2 = "".join(emoji.UNICODE_EMOJI['es'].keys())
emoji3 = "".join(emoji.UNICODE_EMOJI['pt'].keys())
emoji4 = "".join(emoji.UNICODE_EMOJI['it'].keys())
emojis = emoji1+emoji2+emoji3+emoji4

In [160]:
#テキストから取り除く記号を指定
puncs = string.punctuation + "◆▼★①②③④⑤⑥⑦⑧⑨●☆■★【】『』「」、♪"

In [161]:
#全角と半角の処理
def han2zen(txt):
    txt = jctconv.h2z(txt, kana=True, digit=False, ascii=False)
    return jctconv.z2h(txt, kana=False, digit=True, ascii=True)

#記号の削除
def remove_signs(txt):
    rm_signs = emojis + puncs
    for a in rm_signs:
        txt = txt.replace(a, " ")
    return txt

#テキストの整理
def clean_txt(txt):
    txt = han2zen(txt)
    txt = remove_signs(txt)
    txt_list = txt.upper().split()
    txt_list = [x for x in txt_list if len(x) > 1 and re.search(r'[亜-熙ぁ-んァ-ヶa-zA-Z]', x)]
    return " ".join(list(Counter(txt_list)))

In [162]:
new_data['title'] = new_data['title']+" "+new_data['brand']
new_data.drop(columns = ['brand'], inplace=True)
new_data

,title,price,category,delivery_fee,description
0,テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品 Laserdis...,2500,エンタメ/ホビー > エンタメその他 > その他,送料込,テレカ 未使用 50度数 レア 仮面ライダー 誕生20周年 レーザーディスク販売時の販促...
1,送料無料 新品 DVD Perfume WORLD TOUR 1st 初回 Laserdis...,4800,エンタメ/ホビー > エンタメその他 > その他,送料込,新品 DVD Perfume WORLD TOUR 1st 初回プレス盤 ご覧頂きあり...
2,☆ウルトラマン☆レーザーディスク ジャンク？ Laserdiscs (R),1980,エンタメ/ホビー > エンタメその他 > その他,送料込,ウルトラマンのレーザーディスクです 再生機器が無いのと 未開封のようですので現状渡しで...
3,MAISHA （マイシャ）Sadao Watanabe 渡辺貞夫★LD Laserdiscs...,2600,エンタメ/ホビー > エンタメその他 > その他,着払い,ご覧いただきありがとうございます MAISHA マイシャ 1985年 Sadao...
4,LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA Laser...,3000,エンタメ/ホビー > エンタメその他 > その他,着払い,ご覧いただきありがとうございます PARKER S MOOD パーカーズ ムード LI...
...,...,...,...,...,...
56723,コントラスピリッツ、ドラゴンボール スーパーファミコン,2000,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,送料込,２本セットです コントラスピリッツはシール剥がれあります よろしくお願いします
56724,ゲームキューブ 本体＋コントローラー2個＋メモリカード2個＋ソフト2個 ニンテンドーゲーム...,28000,エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体,着払い,箱付きで状態はかなりいいです オレンジ色 マリオカート箱あり バイオハザー...
56725,Gc バイオハザード０ ニンテンドーゲームキューブ,550,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,送料込,ゲームキューブ バイオハザード ゼロ レベッカ ビリーの2人の主人公を切り替えていく...
56726,ゲームキューブ／バイオハザード／8 ニンテンドーゲームキューブ,600,エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト,送料込,ご覧いただきありがとうございます セット内容とコンディション ソフト スレやキ...


In [163]:
#テキストの処理
new_data['title'] = new_data.title.apply(clean_txt)
new_data['category'] = new_data.category.apply(remove_signs)

In [164]:
#カテゴリとブランドのデータタイプをカテゴリにする
new_data['category'] = new_data['category'].astype('category')

In [165]:
new_data

,title,price,category,delivery_fee,description
0,テレカ 未使用品 仮面ライダー 周年 東映ビデオ販売 販促用非売品 LASERDISCS,2500,エンタメ ホビー エンタメその他 その他,送料込,テレカ 未使用 50度数 レア 仮面ライダー 誕生20周年 レーザーディスク販売時の販促...
1,送料無料 新品 DVD PERFUME WORLD TOUR ST 初回 LASERDISCS,4800,エンタメ ホビー エンタメその他 その他,送料込,新品 DVD Perfume WORLD TOUR 1st 初回プレス盤 ご覧頂きあり...
2,ウルトラマン レーザーディスク ジャンク LASERDISCS,1980,エンタメ ホビー エンタメその他 その他,送料込,ウルトラマンのレーザーディスクです 再生機器が無いのと 未開封のようですので現状渡しで...
3,MAISHA マイシャ SADAO WATANABE 渡辺貞夫 LD LASERDISCS,2600,エンタメ ホビー エンタメその他 その他,着払い,ご覧いただきありがとうございます MAISHA マイシャ 1985年 Sadao...
4,LD PARKER MOOD パーカーズ・ムード LIVE AT BRAVA LASERDISCS,3000,エンタメ ホビー エンタメその他 その他,着払い,ご覧いただきありがとうございます PARKER S MOOD パーカーズ ムード LI...
...,...,...,...,...,...
56723,コントラスピリッツ ドラゴンボール スーパーファミコン,2000,エンタメ ホビー テレビゲーム 家庭用ゲームソフト,送料込,２本セットです コントラスピリッツはシール剥がれあります よろしくお願いします
56724,ゲームキューブ 本体 コントローラー メモリカード ソフト ニンテンドーゲームキューブ,28000,エンタメ ホビー テレビゲーム 家庭用ゲーム本体,着払い,箱付きで状態はかなりいいです オレンジ色 マリオカート箱あり バイオハザー...
56725,GC バイオハザード ニンテンドーゲームキューブ,550,エンタメ ホビー テレビゲーム 家庭用ゲームソフト,送料込,ゲームキューブ バイオハザード ゼロ レベッカ ビリーの2人の主人公を切り替えていく...
56726,ゲームキューブ バイオハザード ニンテンドーゲームキューブ,600,エンタメ ホビー テレビゲーム 家庭用ゲームソフト,送料込,ご覧いただきありがとうございます セット内容とコンディション ソフト スレやキ...


In [130]:
#sklearnのライブラリのインポート
import scipy
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from lightgbm import LGBMRegressor
from sklearn.linear_model import Ridge

In [131]:
#商品名をCountVectorizerでベクトル化
count_name = CountVectorizer(min_df=10)
X_name = count_name.fit_transform(new_data["title"])

In [132]:
#カテゴリをCountVectorizerでベクトル化
ctg_array = np.array(new_data['category'].tolist())
count_category = CountVectorizer()
X_category = count_category.fit_transform(ctg_array)

In [133]:
#descriptionの欠損値をNoneに変更
new_data['description'].fillna(value='None', inplace=True)

In [134]:
#descriptionの単語をtfidfでベクトル化
desc_array = np.array(new_data['description'].tolist())
tfidf_descp = TfidfVectorizer(max_features = 10000, ngram_range = (1,3))
X_descp = tfidf_descp.fit_transform(desc_array)

In [135]:
# #ブランド名の二値化
# label_brand = LabelBinarizer(sparse_output=True)
# X_brand = label_brand.fit_transform(new_data["brand"])

In [136]:
#送料負担の有無をダミー変数化
X_dummies = scipy.sparse.csr_matrix(pd.get_dummies(new_data[["delivery_fee"]], sparse = True).values, dtype=int)

In [137]:
#処理を行った特徴量を疎行列にする
X = scipy.sparse.hstack((X_dummies, X_descp, X_category, X_name)).tocsr()

In [138]:
#ターゲットとなる価格を対数変換
target = new_data.loc[:, 'price'].values
target = np.log1p(target)

モデルの作成

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.3, random_state=42)

In [140]:
#Ridge回帰
model = Ridge(solver='auto', fit_intercept=True, alpha=0.5)
Rid = model.fit(X_train, y_train)
preds1 = model.predict(X_test)

In [141]:
def rmsle(Y, Y_pred):
    assert Y.shape == Y_pred.shape
    return np.sqrt(np.mean(np.square(Y_pred - Y)))

def rmsle_lgb(labels, preds):
    return 'rmsle', rmsle(preds, labels), False

In [142]:
#LightGBM
lgbm_params = {'n_estimators': 1500, 'learning_rate': 0.2, 'max_depth': 15,
               'num_leaves': 20, 'subsample': 0.9, 'colsample_bytree': 0.5,
               'n_jobs': 4}

model = LGBMRegressor(**lgbm_params)
LGB = model.fit(X_train, y_train,
          eval_set=[(X_train, y_train)],
          eval_metric=rmsle_lgb,
          early_stopping_rounds=100,
          verbose=True)
preds2 = model.predict(X_test)

[1]	training's l2: 1.63834	training's rmsle: 1.27998
Training until validation scores don't improve for 100 rounds
[2]	training's l2: 1.40432	training's rmsle: 1.18504
[3]	training's l2: 1.24529	training's rmsle: 1.11592
[4]	training's l2: 1.14298	training's rmsle: 1.0691
[5]	training's l2: 1.07714	training's rmsle: 1.03785
[6]	training's l2: 1.02516	training's rmsle: 1.0125
[7]	training's l2: 0.986934	training's rmsle: 0.993446
[8]	training's l2: 0.955954	training's rmsle: 0.977729
[9]	training's l2: 0.931679	training's rmsle: 0.965235
[10]	training's l2: 0.911555	training's rmsle: 0.954754
[11]	training's l2: 0.895281	training's rmsle: 0.946193
[12]	training's l2: 0.881331	training's rmsle: 0.938793
[13]	training's l2: 0.867788	training's rmsle: 0.931551
[14]	training's l2: 0.855642	training's rmsle: 0.925009
[15]	training's l2: 0.843541	training's rmsle: 0.918445
[16]	training's l2: 0.834512	training's rmsle: 0.913516
[17]	training's l2: 0.826587	training's rmsle: 0.909169
[18]	trai

[147]	training's l2: 0.570534	training's rmsle: 0.755337
[148]	training's l2: 0.569734	training's rmsle: 0.754807
[149]	training's l2: 0.569139	training's rmsle: 0.754413
[150]	training's l2: 0.568623	training's rmsle: 0.754071
[151]	training's l2: 0.568068	training's rmsle: 0.753703
[152]	training's l2: 0.567401	training's rmsle: 0.75326
[153]	training's l2: 0.566783	training's rmsle: 0.75285
[154]	training's l2: 0.565972	training's rmsle: 0.752311
[155]	training's l2: 0.565088	training's rmsle: 0.751723
[156]	training's l2: 0.564508	training's rmsle: 0.751337
[157]	training's l2: 0.563896	training's rmsle: 0.75093
[158]	training's l2: 0.563272	training's rmsle: 0.750514
[159]	training's l2: 0.562677	training's rmsle: 0.750118
[160]	training's l2: 0.562069	training's rmsle: 0.749713
[161]	training's l2: 0.561285	training's rmsle: 0.749189
[162]	training's l2: 0.560606	training's rmsle: 0.748736
[163]	training's l2: 0.559998	training's rmsle: 0.74833
[164]	training's l2: 0.559096	train

[292]	training's l2: 0.500109	training's rmsle: 0.707184
[293]	training's l2: 0.499791	training's rmsle: 0.706959
[294]	training's l2: 0.499389	training's rmsle: 0.706674
[295]	training's l2: 0.498908	training's rmsle: 0.706334
[296]	training's l2: 0.498593	training's rmsle: 0.706111
[297]	training's l2: 0.498316	training's rmsle: 0.705915
[298]	training's l2: 0.498025	training's rmsle: 0.705709
[299]	training's l2: 0.497749	training's rmsle: 0.705513
[300]	training's l2: 0.497314	training's rmsle: 0.705205
[301]	training's l2: 0.496782	training's rmsle: 0.704828
[302]	training's l2: 0.496356	training's rmsle: 0.704525
[303]	training's l2: 0.495914	training's rmsle: 0.704212
[304]	training's l2: 0.495653	training's rmsle: 0.704026
[305]	training's l2: 0.495409	training's rmsle: 0.703853
[306]	training's l2: 0.495133	training's rmsle: 0.703657
[307]	training's l2: 0.494851	training's rmsle: 0.703457
[308]	training's l2: 0.49458	training's rmsle: 0.703264
[309]	training's l2: 0.494194	tr

[437]	training's l2: 0.461032	training's rmsle: 0.678994
[438]	training's l2: 0.46087	training's rmsle: 0.678874
[439]	training's l2: 0.460624	training's rmsle: 0.678693
[440]	training's l2: 0.460418	training's rmsle: 0.678541
[441]	training's l2: 0.460223	training's rmsle: 0.678398
[442]	training's l2: 0.460014	training's rmsle: 0.678243
[443]	training's l2: 0.459695	training's rmsle: 0.678008
[444]	training's l2: 0.459504	training's rmsle: 0.677867
[445]	training's l2: 0.459348	training's rmsle: 0.677752
[446]	training's l2: 0.459187	training's rmsle: 0.677634
[447]	training's l2: 0.458947	training's rmsle: 0.677456
[448]	training's l2: 0.45876	training's rmsle: 0.677318
[449]	training's l2: 0.458503	training's rmsle: 0.677129
[450]	training's l2: 0.458337	training's rmsle: 0.677006
[451]	training's l2: 0.45815	training's rmsle: 0.676868
[452]	training's l2: 0.457949	training's rmsle: 0.676719
[453]	training's l2: 0.45775	training's rmsle: 0.676573
[454]	training's l2: 0.457563	train

[582]	training's l2: 0.433459	training's rmsle: 0.658376
[583]	training's l2: 0.433281	training's rmsle: 0.658241
[584]	training's l2: 0.432926	training's rmsle: 0.657971
[585]	training's l2: 0.432731	training's rmsle: 0.657823
[586]	training's l2: 0.432613	training's rmsle: 0.657733
[587]	training's l2: 0.4325	training's rmsle: 0.657647
[588]	training's l2: 0.432351	training's rmsle: 0.657534
[589]	training's l2: 0.432195	training's rmsle: 0.657415
[590]	training's l2: 0.432027	training's rmsle: 0.657288
[591]	training's l2: 0.43188	training's rmsle: 0.657176
[592]	training's l2: 0.431556	training's rmsle: 0.656929
[593]	training's l2: 0.431296	training's rmsle: 0.656731
[594]	training's l2: 0.43108	training's rmsle: 0.656567
[595]	training's l2: 0.430938	training's rmsle: 0.656459
[596]	training's l2: 0.430823	training's rmsle: 0.656371
[597]	training's l2: 0.430649	training's rmsle: 0.656239
[598]	training's l2: 0.430555	training's rmsle: 0.656167
[599]	training's l2: 0.430426	train

...

[877]	training's l2: 0.395933	training's rmsle: 0.629232
[878]	training's l2: 0.395839	training's rmsle: 0.629158
[879]	training's l2: 0.395689	training's rmsle: 0.629038
[880]	training's l2: 0.39558	training's rmsle: 0.628951
[881]	training's l2: 0.395493	training's rmsle: 0.628882
[882]	training's l2: 0.395384	training's rmsle: 0.628795
[883]	training's l2: 0.395312	training's rmsle: 0.628739
[884]	training's l2: 0.395203	training's rmsle: 0.628652
[885]	training's l2: 0.395112	training's rmsle: 0.628579
[886]	training's l2: 0.394997	training's rmsle: 0.628488
[887]	training's l2: 0.394873	training's rmsle: 0.628389
[888]	training's l2: 0.394792	training's rmsle: 0.628324
[889]	training's l2: 0.394552	training's rmsle: 0.628134
[890]	training's l2: 0.394435	training's rmsle: 0.628041
[891]	training's l2: 0.394341	training's rmsle: 0.627966
[892]	training's l2: 0.394255	training's rmsle: 0.627898
[893]	training's l2: 0.394127	training's rmsle: 0.627795
[894]	training's l2: 0.393989	tr

[1022]	training's l2: 0.382534	training's rmsle: 0.618493
[1023]	training's l2: 0.382461	training's rmsle: 0.618434
[1024]	training's l2: 0.38236	training's rmsle: 0.618353
[1025]	training's l2: 0.382236	training's rmsle: 0.618252
[1026]	training's l2: 0.382111	training's rmsle: 0.618151
[1027]	training's l2: 0.382011	training's rmsle: 0.61807
[1028]	training's l2: 0.381867	training's rmsle: 0.617954
[1029]	training's l2: 0.381765	training's rmsle: 0.617871
[1030]	training's l2: 0.381715	training's rmsle: 0.617831
[1031]	training's l2: 0.381648	training's rmsle: 0.617777
[1032]	training's l2: 0.381596	training's rmsle: 0.617735
[1033]	training's l2: 0.381542	training's rmsle: 0.617691
[1034]	training's l2: 0.381488	training's rmsle: 0.617647
[1035]	training's l2: 0.381406	training's rmsle: 0.617581
[1036]	training's l2: 0.381341	training's rmsle: 0.617528
[1037]	training's l2: 0.381245	training's rmsle: 0.61745
[1038]	training's l2: 0.381158	training's rmsle: 0.61738
[1039]	training's 

[1164]	training's l2: 0.371009	training's rmsle: 0.609105
[1165]	training's l2: 0.37092	training's rmsle: 0.609032
[1166]	training's l2: 0.370843	training's rmsle: 0.608969
[1167]	training's l2: 0.37074	training's rmsle: 0.608884
[1168]	training's l2: 0.370649	training's rmsle: 0.60881
[1169]	training's l2: 0.370588	training's rmsle: 0.608759
[1170]	training's l2: 0.3705	training's rmsle: 0.608687
[1171]	training's l2: 0.370411	training's rmsle: 0.608614
[1172]	training's l2: 0.37037	training's rmsle: 0.60858
[1173]	training's l2: 0.370334	training's rmsle: 0.608551
[1174]	training's l2: 0.370291	training's rmsle: 0.608516
[1175]	training's l2: 0.370238	training's rmsle: 0.608472
[1176]	training's l2: 0.370142	training's rmsle: 0.608393
[1177]	training's l2: 0.370062	training's rmsle: 0.608327
[1178]	training's l2: 0.37001	training's rmsle: 0.608284
[1179]	training's l2: 0.369936	training's rmsle: 0.608224
[1180]	training's l2: 0.369887	training's rmsle: 0.608184
[1181]	training's l2: 

[1306]	training's l2: 0.360691	training's rmsle: 0.600576
[1307]	training's l2: 0.360638	training's rmsle: 0.600531
[1308]	training's l2: 0.360594	training's rmsle: 0.600495
[1309]	training's l2: 0.360541	training's rmsle: 0.600451
[1310]	training's l2: 0.360482	training's rmsle: 0.600401
[1311]	training's l2: 0.360398	training's rmsle: 0.600332
[1312]	training's l2: 0.360325	training's rmsle: 0.60027
[1313]	training's l2: 0.360253	training's rmsle: 0.60021
[1314]	training's l2: 0.360198	training's rmsle: 0.600165
[1315]	training's l2: 0.360104	training's rmsle: 0.600086
[1316]	training's l2: 0.360066	training's rmsle: 0.600055
[1317]	training's l2: 0.360008	training's rmsle: 0.600007
[1318]	training's l2: 0.35995	training's rmsle: 0.599958
[1319]	training's l2: 0.359851	training's rmsle: 0.599876
[1320]	training's l2: 0.359765	training's rmsle: 0.599804
[1321]	training's l2: 0.359728	training's rmsle: 0.599774
[1322]	training's l2: 0.359673	training's rmsle: 0.599728
[1323]	training's

[1448]	training's l2: 0.352181	training's rmsle: 0.593449
[1449]	training's l2: 0.352126	training's rmsle: 0.593402
[1450]	training's l2: 0.352062	training's rmsle: 0.593348
[1451]	training's l2: 0.352012	training's rmsle: 0.593306
[1452]	training's l2: 0.351964	training's rmsle: 0.593265
[1453]	training's l2: 0.351929	training's rmsle: 0.593236
[1454]	training's l2: 0.351869	training's rmsle: 0.593185
[1455]	training's l2: 0.351829	training's rmsle: 0.593151
[1456]	training's l2: 0.351781	training's rmsle: 0.593112
[1457]	training's l2: 0.35174	training's rmsle: 0.593077
[1458]	training's l2: 0.351703	training's rmsle: 0.593045
[1459]	training's l2: 0.351632	training's rmsle: 0.592986
[1460]	training's l2: 0.351597	training's rmsle: 0.592956
[1461]	training's l2: 0.351533	training's rmsle: 0.592903
[1462]	training's l2: 0.351428	training's rmsle: 0.592814
[1463]	training's l2: 0.351225	training's rmsle: 0.592642
[1464]	training's l2: 0.351144	training's rmsle: 0.592574
[1465]	training

モデルの評価

In [143]:
rmsle(y_test, preds1)

0.7582484165285666

In [144]:
rmsle(y_test, preds2)

0.7368344133288462

In [145]:
print(f"training dataに対しての精度: {Rid.score(X_train, y_train):.2}")
print(f"test dataに対しての精度: {Rid.score(X_test, y_test):.2f}")

training dataに対しての精度: 0.78
test dataに対しての精度: 0.71


In [146]:
print(f"training dataに対しての精度: {LGB.score(X_train, y_train):.2}")
print(f"test set scoreに対しての精度: {LGB.score(X_test, y_test):.2f}")

training dataに対しての精度: 0.82
test set scoreに対しての精度: 0.73


In [147]:
pre_data = pd.DataFrame({'Rigge': np.expm1(preds1), 'LGB': np.exp(preds2), 'real_price': np.expm1(y_test)})

In [148]:
pre_data

,Rigge,LGB,real_price
0,1144.934033,508.664157,499.0
1,23814.767454,26058.801430,43000.0
2,6242.205353,4918.916101,2300.0
3,11948.386231,9594.442534,3800.0
4,1721.199601,836.955725,700.0
...,...,...,...
22272,1560.829830,1429.361261,1480.0
22273,963.654336,1336.021228,800.0
22274,2512.372155,2793.765922,7200.0
22275,12494.106566,12584.938026,12345.0
